In [1]:
import pandas as pd
import os
import glob
from pathlib import Path
import shutil

def process_ciciot2023_dataset(dataset_folder, output_folder, sample_percentage=0.10):
    """
    پردازش دیتاست CICIoT2023 و نمونه‌برداری 10% از داده‌ها
    فایل‌های اصلی از پوشه dataset خوانده شده و خروجی در پوشه input ذخیره می‌شود
    
    Parameters:
    dataset_folder (str): مسیر پوشه dataset حاوی فایل‌های CSV اصلی
    output_folder (str): مسیر پوشه input برای ذخیره فایل‌های خروجی
    sample_percentage (float): درصد نمونه‌برداری (پیش‌فرض: 0.10 برای 10%)
    """
    
    # ایجاد پوشه خروجی اگر وجود ندارد
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    # پیدا کردن تمام فایل‌های CSV در پوشه dataset
    csv_files = glob.glob(os.path.join(dataset_folder, "*.csv"))
    
    if not csv_files:
        print("هیچ فایل CSVیی در پوشه dataset یافت نشد.")
        return
    
    print(f"تعداد فایل‌های CSV یافت شده در dataset: {len(csv_files)}")
    
    all_samples = []
    file_info = []
    
    # خواندن و پردازش هر فایل از پوشه dataset
    for i, csv_file in enumerate(csv_files, 1):
        try:
            filename = os.path.basename(csv_file)
            print(f"در حال پردازش فایل {i}/{len(csv_files)} از dataset: {filename}")
            
            # خواندن فایل CSV از پوشه dataset
            df = pd.read_csv(csv_file)
            
            # ذخیره اطلاعات اولیه فایل
            original_count = len(df)
            
            # حذف رکوردهای کاملاً خالی
            df_clean = df.dropna(how='all')
            
            # حذف رکوردهای تکراری
            df_clean = df_clean.drop_duplicates()
            
            cleaned_count = len(df_clean)
            
            print(f"  رکوردهای اصلی: {original_count}, پس از پاک‌سازی: {cleaned_count}")
            
            if cleaned_count > 0:
                # نمونه‌برداری 5% از داده‌های پاک‌سازی شده
                sample_size = max(1, int(cleaned_count * sample_percentage))
                df_sample = df_clean.sample(n=sample_size, random_state=42)
                
                # اضافه کردن شناسه فایل منبع
                df_sample['source_file'] = filename
                all_samples.append(df_sample)
                
                file_info.append({
                    'file_name': filename,
                    'original_records': original_count,
                    'cleaned_records': cleaned_count,
                    'sampled_records': len(df_sample)
                })
                
        except Exception as e:
            print(f"خطا در پردازش فایل {csv_file}: {str(e)}")
            continue
    
    if not all_samples:
        print("هیچ داده‌ای برای نمونه‌برداری یافت نشد.")
        return
    
    # ترکیب تمام نمونه‌ها
    combined_samples = pd.concat(all_samples, ignore_index=True)
    
    print(f"\nجمع‌بندی:")
    print(f"تعداد کل نمونه‌های انتخاب شده: {len(combined_samples)}")
    
    # ذخیره در فایل‌های مختلف در پوشه input
    max_records_per_file = 50000  # حداکثر رکورد در هر فایل
    
    if len(combined_samples) <= max_records_per_file:
        # ذخیره در یک فایل
        output_file = os.path.join(output_folder, "ciciot2023_10percent_sample.csv")
        combined_samples.to_csv(output_file, index=False)
        print(f"تمام نمونه‌ها در یک فایل ذخیره شد: {output_file}")
    else:
        # ذخیره در چندین فایل
        num_files = (len(combined_samples) // max_records_per_file) + 1
        
        for i in range(num_files):
            start_idx = i * max_records_per_file
            end_idx = min((i + 1) * max_records_per_file, len(combined_samples))
            
            chunk = combined_samples.iloc[start_idx:end_idx]
            output_file = os.path.join(output_folder, f"ciciot2023_5percent_sample_part_{i+1}.csv")
            chunk.to_csv(output_file, index=False)
            print(f"فایل {i+1} ذخیره شد: {output_file} (تعداد رکورد: {len(chunk)})")
    
    # ذخیره اطلاعات آماری در پوشه input
    info_df = pd.DataFrame(file_info)
    info_file = os.path.join(output_folder, "sampling_info.csv")
    info_df.to_csv(info_file, index=False)
    print(f"اطلاعات آماری ذخیره شد: {info_file}")
    
    # نمایش خلاصه‌ای از داده‌های نمونه‌برداری شده
    print("\n📊 خلاصه داده‌های نمونه‌برداری شده:")
    print(f"تعداد کل نمونه‌ها: {len(combined_samples)}")
    print(f"تعداد ویژگی‌ها: {len(combined_samples.columns)}")
    print(f"توزیع نمونه‌ها بر اساس فایل منبع:")
    source_distribution = combined_samples['source_file'].value_counts()
    for file, count in source_distribution.head(10).items():  # نمایش 10 فایل اول
        print(f"  {file}: {count} نمونه")
    
    if len(source_distribution) > 10:
        print(f"  ... و {len(source_distribution) - 10} فایل دیگر")
    
    return combined_samples

# اجرای تابع
if __name__ == "__main__":
    # مسیر پوشه‌ها
    dataset_folder = "dataset"  # پوشه حاوی فایل‌های اصلی CSV
    output_folder = "input"     # پوشه برای ذخیره فایل‌های خروجی
    
    # بررسی وجود پوشه dataset
    if not os.path.exists(dataset_folder):
        print(f" پوشه {dataset_folder} یافت نشد!")
        print("لطفاً مسیر صحیح پوشه dataset حاوی فایل‌های CSV را مشخص کنید.")
    else:
        # پردازش دیتاست
        result = process_ciciot2023_dataset(
            dataset_folder=dataset_folder,
            output_folder=output_folder,
            sample_percentage=0.10  # 10%
        )
        
        if result is not None:
            print("\n پردازش با موفقیت انجام شد!")
            print(f"فایل‌های اصلی از پوشه '{dataset_folder}' خوانده شدند.")
            print(f"فایل‌های نهایی در پوشه '{output_folder}' ذخیره شدند.")

تعداد فایل‌های CSV یافت شده در dataset: 169
در حال پردازش فایل 1/169 از dataset: part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
  رکوردهای اصلی: 238687, پس از پاک‌سازی: 238687
در حال پردازش فایل 2/169 از dataset: part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
  رکوردهای اصلی: 218805, پس از پاک‌سازی: 218805
در حال پردازش فایل 3/169 از dataset: part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
  رکوردهای اصلی: 275258, پس از پاک‌سازی: 275258
در حال پردازش فایل 4/169 از dataset: part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
  رکوردهای اصلی: 231023, پس از پاک‌سازی: 231023
در حال پردازش فایل 5/169 از dataset: part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
  رکوردهای اصلی: 227491, پس از پاک‌سازی: 227491
در حال پردازش فایل 6/169 از dataset: part-00005-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
  رکوردهای اصلی: 240046, پس از پاک‌سازی: 240046
در حال پردازش فایل 7/169 از dataset: part-00006-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
  رکوردهای